## Project 2 Exploring the Relationship Between SAT Scores and Racial Demographics in NYC Public Schools

In this project, I set out to examine an important—but often overlooked—question in public education:

How do SAT outcomes vary with the racial composition of New York City public schools?

Using publicly available NYC Open Data, I combine school-level demographic information with SAT performance data and conduct a simple exploratory analysis. While this study does not make any causal claims, it offers a descriptive snapshot of patterns that raise deeper questions about equity, opportunity, and structural disparities in education.

In [25]:
import pandas as pd
import plotly.express as px
from IPython.display import HTML

## Data Sources

This project uses two datasets 

School Demographic & Enrollment Data (2005–2012):https://data.cityofnewyork.us/Education/2006-2012-School-Demographics-and-Accountability-S/ihfw-zy9j/about_data
Includes racial composition, enrollment numbers, gender breakdown, ELL status, SPED status, and more.

2012 SAT School-Level Results:https://data.cityofnewyork.us/Education/NYC_SAT/h27v-c5ke/about_data
Contains average scores in Critical Reading, Math Writing and the number of test takers in each school.

After loading the datasets, I restricted the demographic data to the 2011–2012 academic year to align both sources temporally

In [26]:
demo = pd.read_csv('/Users/zhanghongyuan/Downloads/2006_-_2012_School_Demographics_and_Accountability_Snapshot_20251116.csv')
sat=pd.read_csv('/Users/zhanghongyuan/Downloads/2012_SAT_Results.csv')




##Selecting the relevant year

The demographic dataset spans multiple years, so the first step was to filter for 2011–2012, ensuring consistency with the SAT dataset.

In [27]:
demo.head(10)

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,...,75,29.8,149,59.1,7,2.8,149.0,59.1,103.0,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,...,67,32.2,118,56.7,6,2.9,124.0,59.6,84.0,40.4
5,01M015,P.S. 015 ROBERTO CLEMENTE,20102011,,96.5,203,13,37,35,33,...,75,36.9,110,54.2,4,2.0,113.0,55.7,90.0,44.3
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,...,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
7,01M019,P.S. 019 ASHER LEVY,20052006,61.5,NaN,402,15,43,55,53,...,103,25.6,207,51.5,39,9.7,214.0,53.2,188.0,46.8
8,01M019,P.S. 019 ASHER LEVY,20062007,61.5,NaN,312,13,37,45,52,...,70,22.4,172,55.1,19,6.1,157.0,50.3,155.0,49.7
9,01M019,P.S. 019 ASHER LEVY,20072008,61.5,NaN,338,28,48,46,47,...,72,21.3,186,55.0,22,6.5,162.0,47.9,176.0,52.1


In [28]:
#Only keep data in 2017
demo['schoolyear'] = demo['schoolyear'].astype(str).str.strip()
demo_12 = demo[demo['schoolyear'] == '20112012']
demo_12


,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,...,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,P.S. 019 ASHER LEVY,20112012,NaN,61.5,328,32,46,52,54,...,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,PS 020 ANNA SILVER,20112012,NaN,92.5,626,52,102,121,87,...,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3
27,01M034,PS 034 FRANKLIN D ROOSEVELT,20112012,NaN,99.7,401,14,34,38,36,...,90,22.4,275,68.6,8,2.0,204.0,50.9,197.0,49.1
35,01M063,PS 063 WILLIAM MCKINLEY,20112012,NaN,78.9,176,18,20,30,21,...,41,23.3,110,62.5,15,8.5,97.0,55.1,79.0,44.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10046,32K549,BUSHWICK SCHOOL FOR SOCIAL JUSTICE,20112012,NaN,75.1,434,,,,,...,123,28.3,298,68.7,8,1.8,210.0,48.4,224.0,51.6
10053,32K552,ACADEMY OF URBAN PLANNING,20112012,NaN,77.1,398,,,,,...,105,26.4,280,70.4,4,1.0,217.0,54.5,181.0,45.5
10060,32K554,ALL CITY LEADERSHIP SECONDARY SCHOOL,20112012,NaN,81.4,263,,,,,...,34,12.9,209,79.5,4,1.5,137.0,52.1,126.0,47.9
10067,32K556,BUSHWICK LEADERS HIGH SCHOOL FOR ACADEMIC EXCELL,20112012,NaN,88.0,462,,,,,...,98,21.2,357,77.3,0,0.0,246.0,53.2,216.0,46.8


In [29]:
with pd.option_context('display.max_columns', None):
    display(demo_12.head())


,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,grade3,grade4,grade5,grade6,grade7,grade8,grade9,grade10,grade11,grade12,ell_num,ell_percent,sped_num,sped_percent,ctt_num,selfcontained_num,asian_num,asian_per,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,25,28,29,,,,,,,,20.0,10.6,40.0,21.2,23,7,12,6.3,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,P.S. 019 ASHER LEVY,20112012,NaN,61.5,328,32,46,52,54,52,46,46,,,,,,,,33.0,10.1,59.0,18.0,16,16,51,15.5,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,PS 020 ANNA SILVER,20112012,NaN,92.5,626,52,102,121,87,88,85,91,,,,,,,,128.0,20.4,97.0,15.5,49,31,190,30.4,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3
27,01M034,PS 034 FRANKLIN D ROOSEVELT,20112012,NaN,99.7,401,14,34,38,36,45,28,40,55,55,56,,,,,34.0,8.5,106.0,26.4,59,16,22,5.5,90,22.4,275,68.6,8,2.0,204.0,50.9,197.0,49.1
35,01M063,PS 063 WILLIAM MCKINLEY,20112012,NaN,78.9,176,18,20,30,21,31,26,30,,,,,,,,6.0,3.4,45.0,25.6,34,4,9,5.1,41,23.3,110,62.5,15,8.5,97.0,55.1,79.0,44.9


##Merging datasets

To continute the analysis, we merges the two datasets using DBN (District Borough Number) as the common key. This produces a comprehensive dataset with 43–45 columns of demographic and SAT performance information.

In [30]:
# Mearge the datasets with DBN as the common variable
merged = pd.merge(
    demo_12,
    sat,
    on="DBN",
    how="inner"
)


In [31]:
merged.head()



,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,white_per,male_num,male_per,female_num,female_per,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,20112012,NaN,88.6,422,,,,,...,1.7,259.0,61.4,163.0,38.6,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,20112012,NaN,71.8,394,,,,,...,2.3,226.0,57.4,168.0,42.6,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY HIGH SCHOOL,20112012,NaN,71.8,598,,,,,...,10.4,327.0,54.7,271.0,45.3,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,SATELLITE ACADEMY HS @ FORSYTHE STREET,20112012,NaN,72.8,224,,,,,...,3.6,97.0,43.3,127.0,56.7,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE SECONDARY SCHOOL,20112012,NaN,80.7,367,,,,,...,1.6,170.0,46.3,197.0,53.7,MARTA VALLE HIGH SCHOOL,44,390,433,384


In [32]:
merged.columns

Index(['DBN', 'Name', 'schoolyear', 'fl_percent', 'frl_percent',
       'total_enrollment', 'prek', 'k', 'grade1', 'grade2', 'grade3', 'grade4',
       'grade5', 'grade6', 'grade7', 'grade8', 'grade9', 'grade10', 'grade11',
       'grade12', 'ell_num', 'ell_percent', 'sped_num', 'sped_percent',
       'ctt_num', 'selfcontained_num', 'asian_num', 'asian_per', 'black_num',
       'black_per', 'hispanic_num', 'hispanic_per', 'white_num', 'white_per',
       'male_num', 'male_per', 'female_num', 'female_per', 'SCHOOL NAME',
       'Num of SAT Test Takers', 'SAT Critical Reading Avg. Score',
       'SAT Math Avg. Score', 'SAT Writing Avg. Score'],
      dtype='object')

In [33]:
# Convert SAT score columns to numeric and merge total SAT score
sat_cols = [
    'SAT Critical Reading Avg. Score',
    'SAT Math Avg. Score',
    'SAT Writing Avg. Score'
]

for col in sat_cols:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

# Optional: total SAT score
merged['SAT Total Avg Score'] = merged[sat_cols].sum(axis=1)
merged.head()



,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,male_num,male_per,female_num,female_per,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Total Avg Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,20112012,NaN,88.6,422,,,,,...,259.0,61.4,163.0,38.6,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,20112012,NaN,71.8,394,,,,,...,226.0,57.4,168.0,42.6,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY HIGH SCHOOL,20112012,NaN,71.8,598,,,,,...,327.0,54.7,271.0,45.3,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0
3,01M458,SATELLITE ACADEMY HS @ FORSYTHE STREET,20112012,NaN,72.8,224,,,,,...,97.0,43.3,127.0,56.7,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0
4,01M509,MARTA VALLE SECONDARY SCHOOL,20112012,NaN,80.7,367,,,,,...,170.0,46.3,197.0,53.7,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0


## Creating analysis variables

To make the analysis easier:

SAT Reading, Math, and Writing scores were converted to numeric

A total SAT score variable (SAT Total Avg Score) was created by summing the three sections

A non_white_percent variable was calculated as:

In [34]:
merged['non_white_percent'] = 100 - merged['white_per']
merged.head()


,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,male_per,female_num,female_per,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Total Avg Score,non_white_percent
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,20112012,NaN,88.6,422,,,,,...,61.4,163.0,38.6,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,98.3
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,20112012,NaN,71.8,394,,,,,...,57.4,168.0,42.6,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0,97.7
2,01M450,EAST SIDE COMMUNITY HIGH SCHOOL,20112012,NaN,71.8,598,,,,,...,54.7,271.0,45.3,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0,89.6
3,01M458,SATELLITE ACADEMY HS @ FORSYTHE STREET,20112012,NaN,72.8,224,,,,,...,43.3,127.0,56.7,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0,96.4
4,01M509,MARTA VALLE SECONDARY SCHOOL,20112012,NaN,80.7,367,,,,,...,46.3,197.0,53.7,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0,98.4


## Analytical Approach

This analysis focuses on one core descriptive relationship:

How does the percentage of non-white students in a school relate to the school’s average SAT performance?

Before visualizing, I filtered out schools with total SAT scores below 400 (likely data errors or extremely small sample sizes), as shown in the notebook cell on page 5.

The final dataset contains several hundred schools across all five boroughs.

In [35]:
# Keep only schools with SAT Total Avg Score greater than 400 for better visualization
plot_df = merged[merged['SAT Total Avg Score'] > 400].copy()


In [36]:
import plotly.express as px

fig = px.scatter(
    plot_df,
    x='non_white_percent',
    y='SAT Total Avg Score',
    hover_name='Name',  
    title='Relationship Between Percentage of Non-white students and SAT Math Scores (2011–2012)',
    labels={
        '% Non-White': 'Percentage of Students Who Are Non-White',
        'SAT Total Avg Score': 'Average Total SAT Score'
    }
)


fig.show()

HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))


## Key Takeaways
Schools with higher percentages of Non-White students consistently exhibit lower SAT performance, reflecting deep structural inequities and segregation in the NYC school system rather than differences in student ability.

## Interpretation & Caveats

This analysis does not imply causation. Many structural factors sit behind these patterns:

School funding disparities

Neighborhood socioeconomic conditions

Teacher retention and experience

Access to advanced courses

Test preparation resources

Systemic segregation and tracking practices

The data used here captures correlations, not explanations.

Still, the consistent downward slope of the scatter invites deeper inquiry into equity questions:
What would it take for racially diverse and majority-non-white schools to achieve equal opportunities for SAT preparation and college readiness?